In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import os 
os.chdir('/home/oldrain123/MMD/')
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [3]:
import numpy as np
import torch
import pickle
from tqdm.auto import tqdm
from sklearn.utils import check_random_state
from utils_HD import MatConvert, MMDu, TST_MMD_u

In [4]:
def sample_blobs_Q(N1, sigma_mx_2, rows=3, cols=3, rs=None):
    """Generate Blob-D for testing type-II error (or test power)."""
    rs = check_random_state(rs)
    mu = np.zeros(2)
    sigma = np.eye(2) * 0.03
    X = rs.multivariate_normal(mu, sigma, size=N1)
    Y = rs.multivariate_normal(mu, np.eye(2), size=N1)
    # assign to blobs 
    X[:, 0] += rs.randint(rows, size=N1)
    X[:, 1] += rs.randint(cols, size=N1)
    Y_row = rs.randint(rows, size=N1)
    Y_col = rs.randint(cols, size=N1)
    locs = [[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]]
    for i in range(9):
        corr_sigma = sigma_mx_2[i]
        L = np.linalg.cholesky(corr_sigma)
        ind = np.expand_dims((Y_row == locs[i][0]) & (Y_col == locs[i][1]), 1)
        ind2 = np.concatenate((ind, ind), 1)
        Y = np.where(ind2, np.matmul(Y, L) + locs[i], Y)
    return X, Y

In [5]:
class ModelLatentF(torch.nn.Module):
    """Latent space for both domains."""

    def __init__(self, x_in, H, x_out):
        """Init latent features."""
        super(ModelLatentF, self).__init__()
        self.restored = False

        self.latent = torch.nn.Sequential(
            torch.nn.Linear(x_in, H, bias=True),
            torch.nn.Softplus(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Softplus(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Softplus(),
            torch.nn.Linear(H, x_out, bias=True),
        )
    def forward(self, input):
        """Forward the LeNet."""
        fealant = self.latent(input)
        return fealant

In [6]:
# Setup seeds
np.random.seed(1102)
torch.manual_seed(1102)
torch.cuda.manual_seed(1102)
torch.backends.cudnn.deterministic = True
is_cuda = True

In [7]:
# Setup for experiments
dtype = torch.float
device = torch.device("cuda:0")
N_per = 100 # permutation times
alpha = 0.05 # test threshold
n_list = [10, 20, 40, 50, 70, 80, 90, 100] # number of samples in per mode
x_in = 2
H = 50 
x_out = 50
learning_rate = 0.005
N_epoch = 1000 # number of training epochs
K = 10 # number of trails
N = 100 # number of test sets
N_f = 100.0 # number of test sets (float)

In [8]:
# Generate variance and co-variance matrix of Q 
sigma_mx_2_standard = np.array([[0.03, 0], [0, 0.03]])
sigma_mx_2 = np.zeros([9, 2, 2])
for i in range(9):
    sigma_mx_2[i] = sigma_mx_2_standard
    if i < 4:
        sigma_mx_2[i][0 ,1] = -0.02 - 0.002 * i
        sigma_mx_2[i][1, 0] = -0.02 - 0.002 * i
    if i==4:
        sigma_mx_2[i][0, 1] = 0.00
        sigma_mx_2[i][1, 0] = 0.00
    if i>4:
        sigma_mx_2[i][1, 0] = 0.02 + 0.002 * (i-5)
        sigma_mx_2[i][0, 1] = 0.02 + 0.002 * (i-5)

In [9]:
for n in n_list:
    np.random.seed(1102)
    torch.manual_seed(1102)
    torch.cuda.manual_seed(1102)
    N1 = 9 * n
    N2 = 9 * n
    Results = np.zeros([1, K])
    Opt_ep = np.zeros([1, K])
    J_star_u = np.zeros([K, N_epoch])
    ep_OPT = np.zeros([K])
    s_OPT = np.zeros([K])
    s0_OPT = np.zeros([K])
    # Repeat experiments K times (K = 10) and report average test power (rejection rate)
    for kk in tqdm(range(K),desc="Experiment"):
        # Initialize parameters
        if is_cuda:
            model_u = ModelLatentF(x_in, H, x_out).cuda()
        else:
            model_u = ModelLatentF(x_in, H, x_out)
        epsilonOPT = MatConvert(np.random.rand(1) * (10 ** (-10)), device, dtype)
        epsilonOPT.requires_grad = True
        sigmaOPT = MatConvert(np.sqrt(np.random.rand(1) * 0.3), device, dtype)
        sigmaOPT.requires_grad = True
        sigma0OPT = MatConvert(np.sqrt(np.random.rand(1) * 0.002), device, dtype)
        sigma0OPT.requires_grad = True
        # Setup optimizer for training deep kernel
        optimizer_u = torch.optim.Adam(list(model_u.parameters())+[epsilonOPT]+[sigmaOPT]+[sigma0OPT], lr=learning_rate) #
        # Generate Blob-D
        np.random.seed(seed=112 * kk + 1 + n)
        s1,s2 = sample_blobs_Q(N1, sigma_mx_2)
        # REPLACE above line with
        # s1,s2 = sample_blobs(N1)
        # for validating type-I error (s1 ans s2 are from the same distribution)
        if kk==0:
            s1_o = s1
            s2_o = s2
        S = np.concatenate((s1, s2), axis=0)
        S = MatConvert(S, device, dtype)
        # Train deep kernel to maximize test power
        np.random.seed(seed=1102)
        torch.manual_seed(1102)
        torch.cuda.manual_seed(1102)
        for t in range(N_epoch):
            # Compute epsilon, sigma and sigma_0
            ep = torch.exp(epsilonOPT)/(1+torch.exp(epsilonOPT))
            sigma = sigmaOPT ** 2
            sigma0_u = sigma0OPT ** 2
            # Compute output of the deep network
            modelu_output = model_u(S)
            # Compute J (STAT_u)
            TEMP = MMDu(modelu_output, N1, S, sigma, sigma0_u, ep, complete=True)
            mmd_value_temp = -1 * TEMP[0]
            mmd_std_temp = torch.sqrt(TEMP[1] + 10**(-6))
            # STAT_u = torch.div(mmd_value_temp, mmd_std_temp)
            STAT_u = mmd_value_temp - 1e-3 * mmd_std_temp
            J_star_u[kk, t] = STAT_u.item()
            # Initialize optimizer and Compute gradient
            optimizer_u.zero_grad()
            STAT_u.backward(retain_graph=True)
            # Update weights using gradient descent
            optimizer_u.step()
            # Print MMD, std of MMD and J
            if t % 100 == 0:
                print("mmd_value: ", -1 * mmd_value_temp.item(), "mmd_std: ", mmd_std_temp.item(), "Statistic J: ",
                      -1 * STAT_u.item())
        h_u, threshold_u, mmd_value_u = TST_MMD_u(model_u(S), N_per, N1, S, sigma, sigma0_u, ep, alpha, device,
                                                  dtype, complete=True)
        ep_OPT[kk] = ep.item()
        s_OPT[kk] = sigma.item()
        s0_OPT[kk] = sigma0_u.item()
        print(ep, epsilonOPT)
        # Compute test power of deep kernel based MMD
        H_u = np.zeros(N)
        T_u = np.zeros(N)
        M_u = np.zeros(N)
        np.random.seed(1102)
        count_u = 0
        for k in tqdm(range(N), desc="testing"):
            # Generate Blob-D
            np.random.seed(seed=11 * k + 10 + n)
            s1,s2 = sample_blobs_Q(N1, sigma_mx_2)
            # REPLACE above line with
            # s1,s2 = sample_blobs(N1)
            # for validating type-I error (s1 ans s2 are from the same distribution)
            S = np.concatenate((s1, s2), axis=0)
            S = MatConvert(S, device, dtype)
            # Run two sample test (deep kernel) on generated data
            h_u, threshold_u, mmd_value_u = TST_MMD_u(model_u(S), N_per, N1, S, sigma, sigma0_u, ep, alpha, device, dtype, complete=True)
            # Gather results
            count_u = count_u + h_u
            print("MMD-DK:", count_u, "Threshold:", threshold_u, "MMD_value:", mmd_value_u)
            H_u[k] = h_u
            T_u[k] = threshold_u
            M_u[k] = mmd_value_u
        # Print test power of MMD-D
        print("n =",str(n),"--- Test Power of MMD-D: ", H_u.sum()/N_f)
        Results[0, kk] = H_u.sum() / N_f
        print("n =",str(n),"--- Test Power of MMD-D (K times): ",Results[0])
        print("n =",str(n),"--- Average Test Power of MMD-D: ",Results[0].sum()/(kk+1))
    np.save('./Results_Blob_'+str(n)+'_H1_MMD-D',Results)

Experiment:   0%|          | 0/10 [00:00<?, ?it/s]

mmd_value:  0.0021644383668899536 mmd_std:  0.008068447335415618 Statistic J:  0.0021725068142253692
mmd_value:  0.02569808065891266 mmd_std:  0.01532107846386127 Statistic J:  0.02571340173737652
mmd_value:  0.029854312539100647 mmd_std:  0.016583083496523682 Statistic J:  0.02987089562259717
mmd_value:  0.03407007455825806 mmd_std:  0.017724900856652792 Statistic J:  0.03408779945911471
mmd_value:  0.03700682520866394 mmd_std:  0.018605965383057258 Statistic J:  0.037025431174047
mmd_value:  0.03845129907131195 mmd_std:  0.019330698715379714 Statistic J:  0.03847062977002733
mmd_value:  0.042085200548172 mmd_std:  0.020209141459039005 Statistic J:  0.042105409689631036
mmd_value:  0.039398446679115295 mmd_std:  0.019837511583409732 Statistic J:  0.039418284190698705
mmd_value:  0.03010416030883789 mmd_std:  0.016870937067749035 Statistic J:  0.03012103124590564
mmd_value:  0.04197455942630768 mmd_std:  0.02044009816077061 Statistic J:  0.041994999524468446
tensor([0.0394], device='cu

testing:   0%|          | 0/100 [00:00<?, ?it/s]

MMD-DK: 0 Threshold: NaN MMD_value: -0.008449062705039978
MMD-DK: 0 Threshold: NaN MMD_value: 0.00021381676197052002
MMD-DK: 0 Threshold: NaN MMD_value: 0.008842110633850098
MMD-DK: 0 Threshold: NaN MMD_value: 0.001912817358970642
MMD-DK: 0 Threshold: NaN MMD_value: 0.0040021538734436035
MMD-DK: 0 Threshold: NaN MMD_value: -0.0009661614894866943
MMD-DK: 1 Threshold: 0.02418416738510132 MMD_value: 0.0464920699596405
MMD-DK: 1 Threshold: NaN MMD_value: -0.007507100701332092
MMD-DK: 1 Threshold: NaN MMD_value: 0.00815427303314209
MMD-DK: 1 Threshold: NaN MMD_value: 0.005062654614448547
MMD-DK: 1 Threshold: NaN MMD_value: 0.017409056425094604
MMD-DK: 1 Threshold: NaN MMD_value: 0.0005140155553817749
MMD-DK: 1 Threshold: NaN MMD_value: -0.003134489059448242
MMD-DK: 1 Threshold: NaN MMD_value: 0.01273566484451294
MMD-DK: 1 Threshold: NaN MMD_value: 0.005781024694442749
MMD-DK: 1 Threshold: NaN MMD_value: 0.000398293137550354
MMD-DK: 1 Threshold: NaN MMD_value: -0.004544779658317566
MMD-DK: 1

testing:   0%|          | 0/100 [00:00<?, ?it/s]

...

MMD-DK: 1 Threshold: 0.004128612577915192 MMD_value: 0.018065355718135834
MMD-DK: 2 Threshold: 0.0035074204206466675 MMD_value: 0.019366055727005005
MMD-DK: 3 Threshold: 0.004297763109207153 MMD_value: 0.01512889564037323
MMD-DK: 4 Threshold: 0.005193181335926056 MMD_value: 0.020960919559001923
MMD-DK: 5 Threshold: 0.004353910684585571 MMD_value: 0.017710141837596893
MMD-DK: 6 Threshold: 0.00347890704870224 MMD_value: 0.02489527314901352
MMD-DK: 7 Threshold: 0.003453493118286133 MMD_value: 0.0035491138696670532
MMD-DK: 8 Threshold: 0.004900135099887848 MMD_value: 0.008613139390945435
MMD-DK: 9 Threshold: 0.004195801913738251 MMD_value: 0.01975705474615097
MMD-DK: 10 Threshold: 0.003921009600162506 MMD_value: 0.016924604773521423
MMD-DK: 11 Threshold: 0.004122473299503326 MMD_value: 0.02550433576107025
MMD-DK: 12 Threshold: 0.0042385533452034 MMD_value: 0.013584502041339874
MMD-DK: 13 Threshold: 0.003190375864505768 MMD_value: 0.011053025722503662
MMD-DK: 14 Threshold: 0.003669723868370

testing:   0%|          | 0/100 [00:00<?, ?it/s]

MMD-DK: 1 Threshold: 0.0034104660153388977 MMD_value: 0.01311858743429184
MMD-DK: 2 Threshold: 0.003301084041595459 MMD_value: 0.01949930191040039
MMD-DK: 3 Threshold: 0.003031305968761444 MMD_value: 0.01628272980451584
MMD-DK: 4 Threshold: 0.0042790696024894714 MMD_value: 0.01815042644739151
MMD-DK: 5 Threshold: 0.004209801554679871 MMD_value: 0.016544878482818604
MMD-DK: 6 Threshold: 0.0034182965755462646 MMD_value: 0.01893012970685959
MMD-DK: 7 Threshold: 0.0036705955862998962 MMD_value: 0.008723318576812744
MMD-DK: 8 Threshold: 0.003278985619544983 MMD_value: 0.011794373393058777
MMD-DK: 9 Threshold: 0.0029845833778381348 MMD_value: 0.021427735686302185
MMD-DK: 10 Threshold: 0.0028792545199394226 MMD_value: 0.031690217554569244
MMD-DK: 11 Threshold: 0.004136867821216583 MMD_value: 0.02467721700668335
MMD-DK: 12 Threshold: 0.004472583532333374 MMD_value: 0.00730925053358078
MMD-DK: 13 Threshold: 0.0034808292984962463 MMD_value: 0.007569693028926849
MMD-DK: 14 Threshold: 0.0036319196

testing:   0%|          | 0/100 [00:00<?, ?it/s]

MMD-DK: 1 Threshold: 0.0037731677293777466 MMD_value: 0.011976517736911774
MMD-DK: 2 Threshold: 0.00258733332157135 MMD_value: 0.01902085542678833
MMD-DK: 3 Threshold: 0.003913037478923798 MMD_value: 0.009621039032936096
MMD-DK: 4 Threshold: 0.003892563283443451 MMD_value: 0.010570526123046875
MMD-DK: 5 Threshold: 0.003928735852241516 MMD_value: 0.00866546481847763
MMD-DK: 6 Threshold: 0.002929680049419403 MMD_value: 0.015639394521713257
MMD-DK: 7 Threshold: 0.003584429621696472 MMD_value: 0.008973099291324615
MMD-DK: 8 Threshold: 0.0038125887513160706 MMD_value: 0.010064005851745605
MMD-DK: 9 Threshold: 0.004575625061988831 MMD_value: 0.011942990124225616
MMD-DK: 10 Threshold: 0.0028810203075408936 MMD_value: 0.017155282199382782
MMD-DK: 11 Threshold: 0.004234977066516876 MMD_value: 0.013902358710765839
MMD-DK: 12 Threshold: 0.004948072135448456 MMD_value: 0.007109232246875763
MMD-DK: 13 Threshold: 0.0031168311834335327 MMD_value: 0.005146525800228119
MMD-DK: 14 Threshold: 0.004638254